In [ ]:
include("hamiltonian.jl")
using .hamiltonian

using MPSKit, MPSKitModels, TensorKit, TensorOperations

In [ ]:
function calc_groundstate(N, spin; J=1.0, θ=π, max_bond::Int64=10)
    H = AKLT_hamiltonian(J=J, θ=θ, spin=spin)
    Ψ = FiniteMPS(randn, ComplexF64, N, ℂ^2, ℂ^max_bond)
    Ψ, envs, δ = find_groundstate(Ψ, H, DMRG())
    E = sum(expectation_value(Ψ, H))
    println(E)
    return Ψ, E
end

In [ ]:
# spin 1//2 ferromagnetic: good
calc_groundstate(4, 1//2)

# spin 1 heisenberg ferromagnetic: error
calc_groundstate(4, 1)